In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
import math
import time
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch.optim import Adam
import scipy.stats as stats
import statistics
from tqdm.notebook import tqdm
# from tqdm import tqdm
import wandb

## Data Preprocess

### Generate graph

In [2]:
# This class inspired by https://github.com/emschenn/mlg-hw1/blob/master/DrBC.ipynb
class Graph():
    def __init__(self, batch_size: int, min_node: int, max_node: int) -> None:
        """Generate Graph by using networkx function."""
        self.graph_list = []
        for i in range(batch_size):
            g = nx.powerlaw_cluster_graph(n=random.randint(min_node,max_node), m=4, p=0.05)
            self.graph_list.append(g)
    
    def __len__(self) -> int:
        """Get the number of generated graph"""
        return len(self.graph_list)
    
    def get_edge_index(self):
        """Get edge index."""
        src_list, tgt_list, base = [], [], 0
        for g in self.graph_list:
            for src, tgt in g.edges():
                src_list.append(src+base)
                tgt_list.append(tgt+base)
            base += g.number_of_nodes()
        edge_index = [src_list+tgt_list, tgt_list+src_list] # Combining src_list & tgt_list because undirected graph.

        return torch.tensor(edge_index, dtype=torch.long).cuda()

    def get_deg_list(self):
        """Get degree list."""
        deg_list = []
        for g in self.graph_list:
            num_nodes = g.number_of_nodes()
            for i in range(num_nodes):
                deg_list.append([g.degree[i], 1, 1])

        return torch.Tensor(deg_list).cuda()
    
    def get_BC_values(self):
        """Get BC values."""
        BC_value = []
        for g in self.graph_list:         
            BC_value += list(nx.betweenness_centrality(g).values())
        return torch.Tensor(BC_value).cuda()
    
    def get_pair_index(self):
        """Get pair index for doing pairwise ranking loss."""
        list_1, list_2, pair_index = [], [], []
        for g in self.graph_list:
            for i in range(g.number_of_nodes()):
                list_1 += [i, i, i, i, i]
                list_2 += [i, i, i, i, i]
            random.shuffle(list_1)
            random.shuffle(list_2)
            for i, j in zip(list_1, list_2):
                pair_index.append([i, j])
            # initialize list1 & list2
            list_1, list_2 = [], []
            
        return torch.tensor(pair_index, dtype=torch.long).cuda()

### Read test data

In [3]:
# This class inspired by https://github.com/emschenn/mlg-hw1/blob/master/DrBC.ipynb
class read_test_data():
    def __init__(self, data: str='youtube', file_num: int=0, testing: bool=False) -> None:
        """Setting test file path"""
        if data == 'youtube':
            data_path = './hw1_data/youtube/com-youtube.txt'
            data_bc_value = './hw1_data/youtube/com-youtube_score.txt'
        elif data == 'synthetic':
            data_path = './hw1_data/Synthetic/5000/' + str(file_num) + '.txt'
            data_bc_value = './hw1_data/Synthetic/5000/' + str(file_num) + '_score.txt'
        else:
            print('Error!! No such data name. Please try again. \nCurrent data below:')
            print(' - \'youtube\'\n - \'synthetic\' (0-29 file_number)')
            return
        self.testing = testing
        """Opening test file"""
        f_graph = open(data_path, mode='r')
        f_bc_value = open(data_bc_value, mode='r')

        """Reading data"""
        self.bc_value, src_list, tgt_list, self.deg_list, ind = [], [], [], [], 0
        for line in f_bc_value:
            _, val = line.split()
            val = float(val)
            self.bc_value.append([ind, val])
            ind += 1
        num_bc_value = len(self.bc_value)
        for i in range(num_bc_value):
            # initialize
            self.deg_list.append([0, 1, 1])
        for line in f_graph:
            src, tgt = line.split()
            src, tgt = int(src), int(tgt)
            src_list.append(src)
            tgt_list.append(tgt)
            self.deg_list[src][0] += 1
            self.deg_list[tgt][0] += 1
        self.edge_index=[src_list+tgt_list, tgt_list+src_list]

        f_graph.close()
        f_bc_value.close()
    
    def get_edge_index(self):
        if self.testing:
            return torch.tensor(self.edge_index, dtype=torch.long)
        else:
            return torch.tensor(self.edge_index, dtype=torch.long).cuda()

    def get_deg_list(self):
        if self.testing:
            return torch.Tensor(self.deg_list)
        else:
            return torch.Tensor(self.deg_list).cuda()
    
    def get_BC_values(self):
        return self.bc_value

## Model

In [4]:
class DrBC(nn.Module):
    """DrBC model
    They use 5 layers, all embedding dimension are set to be 128.
    """
    def __init__(self) -> None:
        super(DrBC, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(3, 128)
        self.relu1 = nn.LeakyReLU()
        self.norm1 = F.normalize

        self.gcn1 = GCNConv(128, 128)
        self.gru1 = nn.GRU(128, 128)
        self.norm2 = F.normalize

        self.gcn2 = GCNConv(128, 128)
        self.gru2 = nn.GRU(128, 128)
        self.norm3 = F.normalize

        self.gcn3 = GCNConv(128, 128)
        self.gru3 = nn.GRU(128, 128)
        self.norm4 = F.normalize

        self.gcn4 = GCNConv(128, 128)
        self.gru4 = nn.GRU(128, 128)
        self.norm5 = F.normalize

        self.gcn5 = GCNConv(128, 128)
        self.gru5 = nn.GRU(128, 128)
        self.norm6 = F.normalize

        # Decoder
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x, edge_index):
        # Encoder
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.norm1(x, p=2, dim=-1)

        x_gcn = self.gcn1(x, edge_index)
        x1, _ = self.gru1(x_gcn.view(1, *x_gcn.shape), x.view(1, *x.shape))
        x1 = self.norm2(x1, p=2, dim=-1)

        x_gcn = self.gcn2(x1[0], edge_index)
        x2, _ = self.gru2(x_gcn.view(1, *x_gcn.shape), x1)
        x2 = self.norm3(x2, p=2, dim=-1)

        x_gcn = self.gcn3(x2[0], edge_index)
        x3, _ = self.gru3(x_gcn.view(1, *x_gcn.shape), x2)
        x3 = self.norm4(x3, p=2, dim=-1)

        x_gcn = self.gcn4(x3[0], edge_index)
        x4, _ = self.gru4(x_gcn.view(1, *x_gcn.shape), x3)
        x4 = self.norm5(x4, p=2, dim=-1)

        x_gcn = self.gcn5(x4[0], edge_index)
        x5, _ = self.gru5(x_gcn.view(1, *x_gcn.shape), x4)
        x5 = self.norm6(x5, p=2, dim=-1)

        # maxpooling
        l = [x1[0], x2[0], x3[0], x4[0], x5[0]]
        l = torch.stack(l)
        x = torch.max(l, dim = 0).values

        # Decoder        
        x = self.fc2(x)
        x = self.relu2(x)
        output = self.fc3(x)

        return output

## Train & Validate

### train_validate

In [5]:
def train_validate(batch_size: int, 
                   min_node: int, 
                   max_node: int, 
                   num_epoch: int=16, 
                   learning_rate: float=0.0001):
    
    """set hyper-parameters."""
    max_iteration = 1120
    iteration = int(max_iteration / batch_size) # 625 (if batch_size=16 & max_iteration=10000)
    # num_epoch = int(max_iteration / iteration)  # 16  (if batch_size=16 & max_iteration=10000)
    """set training set & validation set."""
    total_train_graph_list = []
    val_g = Graph(100, min_node, max_node)
    """set early-stopping."""
    last_loss = 999999
    patience = 3
    trigger_times = 0

    wandb.init(
        # set the wandb project where this run will be logged
        project = "mlg_hw1",
        # set a run name (otherwise it'll randomly assigned)
        name = str(min_node)+"_"+str(max_node)+"_experiments",
        # track hyperparameters and run metadata
        config = {
            "architecture": "DrBC",
            "dataset": "Synthetic",
            "epochs": num_epoch,
            "batch_size": batch_size,
            "iteration":iteration,
            "learning_rate": learning_rate,
            "min_node": min_node,
            "max_node": max_node,
            })
    config = wandb.config

    """train & validate."""
    model = DrBC()
    model = model.cuda()
    optimizer = Adam(params=model.parameters(), lr=config.learning_rate)
    for epoch in tqdm(range(1, config.epochs+1)):
        # --- Train --------------------------------------------
        model.train()
        for iter in range(1, iteration+1):
            optimizer.zero_grad() # set gradient to zero
            if (iter-1)%5 == 0:
                prev_time = time.time()
            if epoch == 1:
                g = Graph(config.batch_size, min_node, max_node)
                total_train_graph_list.append(g)
            g = total_train_graph_list[iter-1]
            bc = g.get_BC_values()
            outs = model(g.get_deg_list(), g.get_edge_index())
            pair = g.get_pair_index()
            pred = outs[pair[:, 0]] - outs[pair[:, 1]]

            gt = torch.sigmoid(bc[pair[:, 0]] - bc[pair[:, 1]])
            gt = gt.view(-1, 1)
            tr_loss = F.binary_cross_entropy_with_logits(pred, gt, reduction='sum')
            tr_loss.backward()
            optimizer.step() # update model with optimizer
            
            # Show progress
            if iter % 5 == 0 or iter == iteration:
                print('[{}/{} Epoch, {}/{} Iter] Train_loss: {:.8} time:'.format(epoch, num_epoch, iter, iteration, tr_loss.item()),(time.time()-prev_time),"sec")
            train_metrics = {"train_loss": tr_loss,
                             "train_epoch": epoch-1}
            # log train metrics to wandb
            if iter + 1 < iteration: 
                """Use this condition to prevent logging two times train_metrics 
                   when it logs the validation_metrics(write below).
                """
                wandb.log(train_metrics)
            
        # --- Validate --------------------------------------------
        model.eval()
        with torch.no_grad():
            val_bc = val_g.get_BC_values()
            val_outs = model(val_g.get_deg_list(), val_g.get_edge_index())
            val_pair = val_g.get_pair_index()
            val_pred = val_outs[val_pair[:, 0]] - val_outs[val_pair[:, 1]]

            val_gt = torch.sigmoid(val_bc[val_pair[:, 0]] - val_bc[val_pair[:, 1]])
            val_gt = val_gt.view(-1, 1)
            val_loss = F.binary_cross_entropy_with_logits(val_pred, val_gt, reduction='sum')

            # Early-stopping
            cur_val_loss = val_loss
            if epoch == 1:
                last_val_loss = cur_val_loss
                val_metrics = {"val_loss": last_val_loss, 
                               "val_epoch": epoch-1,
                               "earlystop_trigger": trigger_times}
                wandb.log({**train_metrics, **val_metrics})
                print("Epoch[{}/{}] Val_Loss:{:.4f}".format(epoch, num_epoch, cur_val_loss.item()), '  trigger times:', trigger_times)
                continue
            if cur_val_loss >= last_val_loss:
                trigger_times += 1
                if trigger_times >= patience:
                    print("Epoch[{}/{}] Val_Loss:{:.4f}".format(epoch, num_epoch, cur_val_loss.item()), '  trigger times:', trigger_times)
                    print('Early Stopping!!\nStart to test process.')
                    # torch.save(model.state_dict(), "./weight.pth")
                    return model
            else:
                trigger_times = 0
            last_val_loss = cur_val_loss

        validation_metrics = {"val_loss": last_val_loss,
                              "val_epoch": epoch-1,
                              "earlystop_trigger": trigger_times}
        # log train & validate metrics to wandb
        wandb.log({**train_metrics, **validation_metrics})
        print("Epoch[{}/{}] Val_Loss:{:.4f}".format(epoch, num_epoch, last_val_loss.item()), '  trigger times:', trigger_times)
    # torch.save(model.state_dict(), "./weight.pth")
    wandb.finish()
    return model

## Evaluation Metrics

In [6]:
# Evaluation metrics
def TopN_accuracy(outs, gt_data, n: int):
    pred_bc_val, gt_bc_val = [], []
    for i, j in enumerate(outs.tolist()):
        pred_bc_val.append([i, *j])
    gt_bc_val = gt_data.get_BC_values()

    tmp_gt_bcval = gt_bc_val.copy()
    tmp_pred_bcval =pred_bc_val.copy()
    tmp_gt_bcval.sort(key=lambda element: element[1], reverse=True)
    tmp_pred_bcval.sort(key=lambda element: element[1], reverse=True)
    
    pred, gt = [], []
    for i in range(int(len(tmp_pred_bcval)*n/100)):
        pred.append(tmp_pred_bcval[i][0])
        gt.append(tmp_gt_bcval[i][0])
    
    return float(len(set(gt)&set(pred)) / len(pred))

def Kendall_tau_distance(outs, gt_data):
    pred_bc_val, gt_bc_val = [], []
    for i, j in enumerate(outs.tolist()):
        pred_bc_val.append(*j)
    for i in gt_data.get_BC_values():
        gt_bc_val.append(i[1])
    tau, _ = stats.kendalltau(pred_bc_val, gt_bc_val)
    
    return float(tau)

## Testing

In [7]:
def test(model, test_data):
    model.eval()
    model.cpu()
    with torch.no_grad():
        top1_list = []
        top5_list = []
        top10_list = []
        kendall_ist = []
        wall_clock_time_list = []

        for data in test_data:
            test_deg = data.get_deg_list()
            test_edge_ind = data.get_edge_index()
            t0 = time.time()
            outs = model(test_deg, test_edge_ind)
            wall_clock_time = time.time() - t0

            top1_list.append(TopN_accuracy(outs, data, 1))
            top5_list.append(TopN_accuracy(outs, data, 5))
            top10_list.append(TopN_accuracy(outs, data, 10))
            kendall_ist.append(Kendall_tau_distance(outs, data))
            wall_clock_time_list.append(wall_clock_time)

    # show result
    if len(test_data) == 1:
        print("Wall clock time:", sum(wall_clock_time_list) / len(wall_clock_time_list))
        print("top-1%:", sum(top1_list) / len(top1_list))
        print("top-5%:", sum(top5_list) / len(top5_list))
        print("top-10%:", sum(top10_list) / len(top10_list))
        print("kendall tau disance:",sum(kendall_ist) / len(kendall_ist),"\n")
    else:
        print("Wall clock time:", sum(wall_clock_time_list) / len(wall_clock_time_list),'+-',statistics.stdev(wall_clock_time_list))
        print("top-1%:", sum(top1_list) / len(top1_list),'+-',statistics.stdev(top1_list))
        print("top-5%:", sum(top5_list) / len(top5_list),'+-',statistics.stdev(top5_list))
        print("top-10%:", sum(top10_list) / len(top10_list),'+-',statistics.stdev(top10_list))
        print("kendall tau disance:",sum(kendall_ist) / len(kendall_ist),'+-',statistics.stdev(kendall_ist),"\n")

In [ ]:
batch_size = 16
min_node = 100
max_node = 200

model = train_validate(batch_size, min_node, max_node, 1)
torch.save(model, './models/testing_model.pt')
model = torch.load('./models/testing_model.pt')

# 'synthetic' test data
test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i, testing=True)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)

# 'youtube' test data
test_data = []
tmp_data = read_test_data('youtube', testing=True)
test_data.append(tmp_data)
print("Test in youtube data:")
test(model, test_data)

## Experiments

### node：100-200

In [8]:
batch_size = 32
min_node = 100
max_node = 200
lr = 0.00008

model = train_validate(batch_size, min_node, max_node, 16, lr)
torch.save(model, './models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: e19886893. Use `wandb login --relogin` to force relogin


  0%|          | 0/16 [00:00<?, ?it/s]

[1/16 Epoch, 5/35 Iter] Train_loss: 16724.232 time: 11.325495481491089 sec
[1/16 Epoch, 10/35 Iter] Train_loss: 16629.717 time: 11.66914415359497 sec
[1/16 Epoch, 15/35 Iter] Train_loss: 17381.773 time: 11.528548955917358 sec
[1/16 Epoch, 20/35 Iter] Train_loss: 16559.875 time: 11.606628656387329 sec
[1/16 Epoch, 25/35 Iter] Train_loss: 16847.801 time: 11.216149806976318 sec
[1/16 Epoch, 30/35 Iter] Train_loss: 16473.137 time: 11.122397422790527 sec
[1/16 Epoch, 35/35 Iter] Train_loss: 17886.832 time: 11.341087102890015 sec
Epoch[1/16] Val_Loss:53036.2578   trigger times: 0
[2/16 Epoch, 5/35 Iter] Train_loss: 16722.451 time: 11.059908151626587 sec
[2/16 Epoch, 10/35 Iter] Train_loss: 16628.512 time: 11.184850454330444 sec
[2/16 Epoch, 15/35 Iter] Train_loss: 17381.047 time: 11.262988090515137 sec
[2/16 Epoch, 20/35 Iter] Train_loss: 16559.408 time: 11.559818029403687 sec
[2/16 Epoch, 25/35 Iter] Train_loss: 16847.502 time: 11.10674524307251 sec
[2/16 Epoch, 30/35 Iter] Train_loss: 1647

earlystop_trigger,▁▁▁▁▅▁▁▅█▁▅▁▁▅█▁
train_epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train_loss,▄▁▁▆▆▆▄▄▄▄▄▁▁▆▃▆▄▄▄▄▄▄▁▆▃▆▄▄▆▄▄▄▁▆▃▃▄▄▆█
val_epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
val_loss,█▄▁▁▂▂▂▂▃▂▅▃▃▄▅▃
earlystop_trigger,0
train_epoch,15
train_loss,17886.09766
val_epoch,15
val_loss,53035.83594


In [11]:
batch_size = 32
min_node = 100
max_node = 200
lr = 0.00008
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')

test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i, testing=True)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)

Test in synthetic data:
Wall clock time: 0.13851188818613688 +- 0.012798659712158317
top-1%: 0.9033333333333338 +- 0.029749915483314692
top-5%: 0.8412000000000001 +- 0.02074874321079679
top-10%: 0.8084000000000002 +- 0.01731971171218223
kendall tau disance: 0.4443821200059742 +- 0.012122740177771667 



In [12]:
batch_size = 32
min_node = 100
max_node = 200
lr = 0.00008
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')
# 'youtube' test data
test_data = []
tmp_data = read_test_data('youtube', testing=True)
test_data.append(tmp_data)
print("Test in youtube data:")
test(model, test_data)

Test in youtube data:
Wall clock time: 29.149409532546997
top-1%: 0.6817060274938315
top-5%: 0.6162942337515861
top-10%: 0.5763025491457322
kendall tau disance: 0.10276323958690384 



### node：200-300

In [8]:
batch_size = 32
min_node = 200
max_node = 300
lr = 0.00008

model = train_validate(batch_size, min_node, max_node, 16, lr)
torch.save(model, './models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: e19886893. Use `wandb login --relogin` to force relogin


  0%|          | 0/16 [00:00<?, ?it/s]

[1/16 Epoch, 5/35 Iter] Train_loss: 28067.969 time: 30.258564710617065 sec
[1/16 Epoch, 10/35 Iter] Train_loss: 27588.439 time: 28.790122270584106 sec
[1/16 Epoch, 15/35 Iter] Train_loss: 28478.238 time: 30.03984570503235 sec
[1/16 Epoch, 20/35 Iter] Train_loss: 27023.113 time: 29.149421453475952 sec
[1/16 Epoch, 25/35 Iter] Train_loss: 26971.342 time: 27.77471351623535 sec
[1/16 Epoch, 30/35 Iter] Train_loss: 27718.695 time: 29.274418592453003 sec
[1/16 Epoch, 35/35 Iter] Train_loss: 27652.219 time: 28.946316242218018 sec
Epoch[1/16] Val_Loss:86555.8125   trigger times: 0
[2/16 Epoch, 5/35 Iter] Train_loss: 28065.834 time: 29.78988027572632 sec
[2/16 Epoch, 10/35 Iter] Train_loss: 27587.512 time: 28.524581909179688 sec
[2/16 Epoch, 15/35 Iter] Train_loss: 28477.52 time: 29.680545568466187 sec
[2/16 Epoch, 20/35 Iter] Train_loss: 27022.553 time: 28.587053775787354 sec
[2/16 Epoch, 25/35 Iter] Train_loss: 26970.533 time: 27.72788119316101 sec
[2/16 Epoch, 30/35 Iter] Train_loss: 27718.0

earlystop_trigger,▁▁▁▁▅▁▁▁▅█▁▁▅▁▁▅
train_epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train_loss,▅▁▂▃▃▇▁█▃▄▅▁▂▃▃▇▁█▃▄▅▁▂▃▃▇▁█▄▄▅▁▂▃▃█▁█▄▃
val_epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
val_loss,█▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁
earlystop_trigger,1
train_epoch,15
train_loss,27650.87109
val_epoch,15
val_loss,86554.26562


In [9]:
batch_size = 32
min_node = 200
max_node = 300
lr = 0.00008
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')

test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i, testing=True)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)

Test in synthetic data:
Wall clock time: 0.1385127305984497 +- 0.015745694808447227
top-1%: 0.9140000000000005 +- 0.02736723437326794
top-5%: 0.8588 +- 0.019935067006451843
top-10%: 0.8289999999999998 +- 0.018418506435385697
kendall tau disance: 0.4978663197460111 +- 0.01780422318074113 



In [10]:
batch_size = 32
min_node = 200
max_node = 300
lr = 0.00008
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')
# 'youtube' test data
test_data = []
tmp_data = read_test_data('youtube', testing=True)
test_data.append(tmp_data)
print("Test in youtube data:")
test(model, test_data)

Test in youtube data:
Wall clock time: 29.477470874786377
top-1%: 0.6691046880507578
top-5%: 0.4978147469335965
top-10%: 0.35267735199006073
kendall tau disance: -0.2657820805851588 



c:\ProgramData\Anaconda3\envs\ML\lib\site-packages\scipy\stats\_stats_py.py:5278: RuntimeWarning: overflow encountered in scalar multiply
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


### node：1000-1200

In [14]:
batch_size = 32
min_node = 1000
max_node = 1200
lr = 0.00008

model = train_validate(batch_size, min_node, max_node, 16, lr)
torch.save(model, './models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')

  0%|          | 0/16 [00:00<?, ?it/s]

[1/16 Epoch, 5/35 Iter] Train_loss: 124479.59 time: 562.4151163101196 sec
[1/16 Epoch, 10/35 Iter] Train_loss: 123505.82 time: 558.1661126613617 sec
[1/16 Epoch, 15/35 Iter] Train_loss: 120209.86 time: 542.7634718418121 sec
[1/16 Epoch, 20/35 Iter] Train_loss: 121952.95 time: 554.7762820720673 sec
[1/16 Epoch, 25/35 Iter] Train_loss: 122057.77 time: 564.6646175384521 sec
[1/16 Epoch, 30/35 Iter] Train_loss: 123145.45 time: 556.1821825504303 sec
[1/16 Epoch, 35/35 Iter] Train_loss: 121557.05 time: 558.8066141605377 sec
Epoch[1/16] Val_Loss:379047.6875   trigger times: 0
[2/16 Epoch, 5/35 Iter] Train_loss: 124478.27 time: 576.5680804252625 sec
[2/16 Epoch, 10/35 Iter] Train_loss: 123505.09 time: 567.8669941425323 sec
[2/16 Epoch, 15/35 Iter] Train_loss: 120209.56 time: 552.1206548213959 sec
[2/16 Epoch, 20/35 Iter] Train_loss: 121952.64 time: 559.8375961780548 sec
[2/16 Epoch, 25/35 Iter] Train_loss: 122057.66 time: 570.2882454395294 sec
[2/16 Epoch, 30/35 Iter] Train_loss: 123145.41 tim

earlystop_trigger,▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁
train_epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train_loss,▇█▃▄█▆▅▆█▇▇█▃▄▇▆▅▆█▇▇▆▃▄▇▆▅▆▄▇▇▆▃▄▇▁▅▆▄▅
val_epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
val_loss,█▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁
earlystop_trigger,0
train_epoch,15
train_loss,121556.90625
val_epoch,15
val_loss,379046.1875


In [20]:
batch_size = 32
min_node = 1000
max_node = 1200
lr = 0.00008
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')

test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i, testing=True)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)

Test in synthetic data:
Wall clock time: 0.15204795201619467 +- 0.016881041557345253
top-1%: 0.9286666666666671 +- 0.025014938065819396
top-5%: 0.8790666666666669 +- 0.018515479699585096
top-10%: 0.8595333333333335 +- 0.016845614047688456
kendall tau disance: 0.6591245288626779 +- 0.016678040660441186 



In [21]:
batch_size = 32
min_node = 1000
max_node = 1200
lr = 0.00008
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_00008model.pt')
# 'youtube' test data
test_data = []
tmp_data = read_test_data('youtube', testing=True)
test_data.append(tmp_data)
print("Test in youtube data:")
test(model, test_data)

Test in youtube data:
Wall clock time: 28.883826732635498
top-1%: 0.6913993655269651
top-5%: 0.35154377555336247
top-10%: 0.21886702676030276
kendall tau disance: -0.3616086335279728 



### node：1000-1200, batch=32

In [37]:
batch_size = 32
min_node = 1000
max_node = 1200
lr=0.0005

model = train_validate(batch_size, min_node, max_node, 16, lr)
torch.save(model, './models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

  0%|          | 0/16 [00:00<?, ?it/s]

[1/16 Epoch, 5/35 Iter] Train_loss: 122421.59 time: 547.371741771698 sec
[1/16 Epoch, 10/35 Iter] Train_loss: 122494.53 time: 546.6688342094421 sec
[1/16 Epoch, 15/35 Iter] Train_loss: 124340.7 time: 556.9007573127747 sec
[1/16 Epoch, 20/35 Iter] Train_loss: 120275.1 time: 571.0693392753601 sec
[1/16 Epoch, 25/35 Iter] Train_loss: 120486.72 time: 551.855122089386 sec
[1/16 Epoch, 30/35 Iter] Train_loss: 122254.24 time: 559.7594583034515 sec
[1/16 Epoch, 35/35 Iter] Train_loss: 119925.39 time: 560.7123913764954 sec
Epoch[1/16] Val_Loss:379098.9375
[2/16 Epoch, 5/35 Iter] Train_loss: 122420.73 time: 559.2283911705017 sec
[2/16 Epoch, 10/35 Iter] Train_loss: 122493.97 time: 556.6820895671844 sec
[2/16 Epoch, 15/35 Iter] Train_loss: 124340.45 time: 562.6494407653809 sec
[2/16 Epoch, 20/35 Iter] Train_loss: 120274.72 time: 562.8837523460388 sec
[2/16 Epoch, 25/35 Iter] Train_loss: 120486.61 time: 556.2290372848511 sec
[2/16 Epoch, 30/35 Iter] Train_loss: 122254.22 time: 562.6806797981262 se

In [39]:
batch_size = 32
min_node = 1000
max_node = 1200
lr=0.0005
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i, testing=True)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)

Test in synthetic data:
Wall clock time: 0.1359073797861735 +- 0.01240942738964881
top-1%: 0.9280000000000006 +- 0.02605034912731826
top-5%: 0.8784000000000003 +- 0.01790703964829245
top-10%: 0.8614666666666667 +- 0.0174903585920404
kendall tau disance: 0.6886484222564367 +- 0.01889704099299993 



In [40]:
batch_size = 32
min_node = 1000
max_node = 1200
lr=0.0005
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')
# 'youtube' test data
test_data = []
tmp_data = read_test_data('youtube', testing=True)
test_data.append(tmp_data)
print("Test in youtube data:")
test(model, test_data)

Test in youtube data:
Wall clock time: 31.19578528404236
top-1%: 0.6979203383856186
top-5%: 0.3965000704920344
top-10%: 0.2079584805575871
kendall tau disance: -0.43096769409637614 



c:\ProgramData\Anaconda3\envs\ML\lib\site-packages\scipy\stats\_stats_py.py:5278: RuntimeWarning: overflow encountered in scalar multiply
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


### node：1000-1200, batch=32, lr=0.00005

In [ ]:
batch_size = 32
min_node = 1000
max_node = 1200
lr=0.00005

model = train_validate(batch_size, min_node, max_node, 16, lr)
torch.save(model, './models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

In [ ]:
batch_size = 32
min_node = 1000
max_node = 1200
lr=0.00005
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i, testing=True)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)

In [ ]:
batch_size = 32
min_node = 1000
max_node = 1200
lr=0.00005
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')
# 'youtube' test data
test_data = []
tmp_data = read_test_data('youtube', testing=True)
test_data.append(tmp_data)
print("Test in youtube data:")
test(model, test_data)

## (Not done) experiments (cost too much time...)

### node：2000-3000

In [16]:
batch_size = 32
min_node = 2000
max_node = 3000
lr = 0.00005

model = train_validate(batch_size, min_node, max_node, 16, lr)
torch.save(model, './models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

  0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
batch_size = 32
min_node = 2000
max_node = 3000
lr = 0.00005
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)

In [ ]:
batch_size = 32
min_node = 2000
max_node = 3000
lr = 0.00005
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')
# 'youtube' test data
test_data = []
tmp_data = read_test_data('youtube', testing=True)
test_data.append(tmp_data)
print("Test in youtube data:")
test(model, test_data)

### node：4000-5000

In [ ]:
batch_size = 16
min_node = 4000
max_node = 5000

model = train_validate(batch_size, min_node, max_node)
torch.save(model, './models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

In [ ]:
batch_size = 16
min_node = 4000
max_node = 5000
model = torch.load('./models/_'+str(batch_size)+'_'+str(min_node)+'_'+str(max_node)+'_model.pt')

test_data = []
for i in range(30):
    tmp_data = read_test_data('synthetic', i)
    test_data.append(tmp_data)

print("Test in synthetic data:")
test(model, test_data)